In [ ]:
#Implement convolution operation for a sample image of shape (H=6, W=6, C=1) with a
#random kernel of size (3,3) using torch.nn.functional.conv2d.What is the dimension of the output image? Apply, various values for parameter stride=1
'''and note the change in the dimension of the output image. Arrive at an equation for the
output image size with respect to the kernel size and stride and verify your answer with  code. Now, repeat the exercise by changing padding parameter. Obtain a formula using
kernel, stride, and padding to get the output image size. What is the total number of
parameters in your network? Verify with code.'''

In [8]:
import torch 
import torch.nn.functional as F
img = torch.rand(6,6)
print(img)
img = torch.unsqueeze(img,dim=0)
print(f"image shape: {img.shape}")
print(img)
kernel = torch.ones(3,3)
print(f"kernel: {kernel}")
kernel = torch.unsqueeze(img,dim=0)
kernel = torch.unsqueeze(img,dim=0)
outimage = F.conv2d(img, kernel, stride=1, padding=1)
print("outimage=", outimage)
for padding in range(4):  # Try paddings from 0 to 3
    out_image = F.conv2d(img, kernel, stride=1, padding=padding)
    print(f"Output image shape with padding={padding}: {out_image.shape}")

# Formula for output image size with respect to kernel size, stride, and padding:
# output_size = ((input_size - kernel_size + 2 * padding) / stride) + 1

# Calculate total number of parameters in the network
total_params = kernel.numel()  # Number of parameters in the kernel
print(f"Total number of parameters in the network: {total_params}")

tensor([[0.7052, 0.0011, 0.6545, 0.7284, 0.9724, 0.8017],
        [0.2665, 0.7070, 0.1184, 0.3428, 0.3115, 0.1859],
        [0.8497, 0.3080, 0.7402, 0.7722, 0.7764, 0.8089],
        [0.7511, 0.1458, 0.4076, 0.7078, 0.2020, 0.8551],
        [0.0653, 0.4380, 0.0450, 0.7740, 0.3332, 0.7223],
        [0.0494, 0.6175, 0.3621, 0.0624, 0.5676, 0.6661]])
image shape: torch.Size([1, 6, 6])
tensor([[[0.7052, 0.0011, 0.6545, 0.7284, 0.9724, 0.8017],
         [0.2665, 0.7070, 0.1184, 0.3428, 0.3115, 0.1859],
         [0.8497, 0.3080, 0.7402, 0.7722, 0.7764, 0.8089],
         [0.7511, 0.1458, 0.4076, 0.7078, 0.2020, 0.8551],
         [0.0653, 0.4380, 0.0450, 0.7740, 0.3332, 0.7223],
         [0.0494, 0.6175, 0.3621, 0.0624, 0.5676, 0.6661]]])
kernel: tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
outimage= tensor([[[ 5.8347,  6.9093,  5.1968],
         [ 6.7775, 11.8359,  6.7775],
         [ 5.1968,  6.9093,  5.8347]]])
Output image shape with padding=0: torch.Size([1, 1, 1])
Ou

In [ ]:
'''Apply torch.nn.Conv2d to the input image of Qn 1 with out-channel=3 and observe the
output. Implement the equivalent of torch.nn.Conv2d using the torch.nn.functional.conv2D
to get the same output. You may ignore bias'''

In [9]:
import torch
import torch.nn as nn


img = torch.rand(1, 1, 6, 6)  
conv_layer = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3)
output_conv = conv_layer(img)
print("Output shape using torch.nn.Conv2d:", output_conv.shape)
kernel_weights = torch.randn(3, 1, 3, 3)
output_conv_func = torch.nn.functional.conv2d(img, kernel_weights, padding=0)
print("Output shape using torch.nn.functional.conv2d:", output_conv_func.shape)


Output shape using torch.nn.Conv2d: torch.Size([1, 3, 4, 4])
Output shape using torch.nn.functional.conv2d: torch.Size([1, 3, 4, 4])


In [ ]:
'''Implement CNN for classifying digits in MNIST dataset using PyTorch. Display the
classification accuracy in the form of a Confusion matrix. Verify the number of learnable
parameters in the model.
Training a CNN on an image dataset is similar to training a basic multi-layer feed-forward
network on numerical data as outlined below.
Define model architecture
Load dataset from disk
Loop over epochs and batches
Make predictions and compute loss
Properly zero our gradient, perform backpropagation, and update model parameters'''

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

# Define the CNN model
class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(128, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2)
        )

        self.clf_head = nn.Sequential(nn.Linear(64, 20),
                                      nn.ReLU(),
                                      nn.Linear(20, 10))

    def forward(self, x):
        features = self.net(x)
        return self.clf_head(features.view(x.size(0), -1))

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)


batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

model = CNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


num_epochs = 5
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))


model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

# Plot confusion matrix
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of learnable parameters in the model: {total_params}')


Epoch [1/5], Step [100/600], Loss: 0.4720
Epoch [1/5], Step [200/600], Loss: 0.2619
Epoch [1/5], Step [300/600], Loss: 0.2023


In [ ]:
'''Modify CNN of Qn. 3 to reduce the number of parameters in the network. Draw a plot of
percentage drop in parameters vs accuracy.'''

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

class ReducedCNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),  
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(32, 64, kernel_size=3),  
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            nn.Conv2d(64, 32, kernel_size=3),  
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2)
        )

        self.clf_head = nn.Sequential(nn.Linear(32, 20), 
                                      nn.ReLU(),
                                      nn.Linear(20, 10))

    def forward(self, x):
        features = self.net(x)
        return self.clf_head(features.view(x.size(0), -1))

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

model = ReducedCNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

# Plot confusion matrix
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of learnable parameters in the model: {total_params}')
